In [ ]:
pip install --upgrade pip
pip install --upgrade transformers datasets[audio] accelerate

In [ ]:
import torch
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
#from datasets import load_dataset


device = "cuda:0" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

model_id = "openai/whisper-large-v3-turbo"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True
)
model.to(device)

processor = AutoProcessor.from_pretrained(model_id)

pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    torch_dtype=torch_dtype,
    device=device,
    return_timestamps=True, # mp3파일을 청크단위로 쪼갬
    chunk_length_s=10,
    stride_length_s=2, # 청크가 겹치는 구간(2초)
)

#dataset = load_dataset("distil-whisper/librispeech_long", "clean", split="validation")
#sample = dataset[0]["audio"]
sample = "./CH/chapter5/data/lsy_audio_2023_58s.mp3"

result = pipe(sample)
print(result)

start_end_text = []

for chunk in result["chunks"]:
    start = chunk["timestamp"][0] # 청크의 시작 시간
    end = chunk["timestamp"][1] # 청크의 끝 시간
    text = chunk["text"] # 청크의 텍스트
    start_end_text.append((start, end, text))

import pandas as pd

df = pd.DataFrame(start_end_text, columns=["start", "end", "text"])
df.to_csv("whisper_huggingface_output.csv", index=False, sep=":")
display(df)
